# Efficinet Gap

### Initialization: libraries and initial settings

In [1]:
# ----- Load libraries -----

# Built-in
using Colors, CSV, Dates, DataFrames, Distributions, FileIO, JLD2, XLSX, KernelDensity, PlotlyJS;
pltjs = PlotlyJS

# Custom
include("./code/Metropolis-Within-Gibbs/MetropolisWithinGibbs.jl");
using Main.MetropolisWithinGibbs;

┌ Warning: Kaledio is not available on this system. Julia will be unable to produce any plots.
└ @ PlotlyBase C:\Users\marti\.julia\packages\PlotlyBase\NxSlF\src\kaleido.jl:58


In [2]:
# ----- Load results from output file -----

res           = load("$(pwd())/res_efficient.jld2");
nDraws        = res["nDraws"];
burnin        = res["burnin"];
σʸ            = res["σʸ"]';
date          = res["date"];
data          = res["data"] .* σʸ';
distr_α       = res["distr_α"];
chain_θ_bound = res["chain_θ_bound"]; # d, Z, R, c, T, Q, λ, ρ, total
MNEMONIC      = res["MNEMONIC"];



data[ismissing.(data)] .= NaN;

par_ind  = res["par_ind"];


par_size = SizeParSsm(sum(par_ind.d),
                                 sum(sum(par_ind.Z)),
                                 sum(sum(par_ind.Z_plus)),
                                 sum(sum(par_ind.Z_minus)),
                                 sum(sum(par_ind.R)),
                                 sum(par_ind.c),
                                 sum(sum(par_ind.T)),
                                 sum(sum(par_ind.Q)),
                                 sum(sum(par_ind.Q_cov)), 
                                 sum(sum(par_ind.λ)),
                                 sum(sum(par_ind.ρ)),
                                 sum(par_ind.d) + sum(sum(par_ind.Z)) + sum(sum(par_ind.Z_plus)) +
                                    sum(sum(par_ind.Z_minus)) + sum(sum(par_ind.R)) +
                                    sum(par_ind.c) + sum(sum(par_ind.T)) + sum(sum(par_ind.Q)) +
                                    sum(sum(par_ind.Q_cov)) + sum(sum(par_ind.λ)) + sum(sum(par_ind.ρ)));





## Load States and data

In [3]:
y = data[:, 1];
e_data = data[:, 2];
u_data = data[:, 3];

α_med = mapslices(x -> quantile(x, 0.5), distr_α, dims=3)[:,:,1];
α_05 = mapslices(x -> quantile(x, 0.025), distr_α, dims=3)[:,:,1];
α_16 = mapslices(x -> quantile(x, 0.16), distr_α, dims=3)[:,:,1];
α_84 = mapslices(x -> quantile(x, 0.84), distr_α, dims=3)[:,:,1];
α_95 = mapslices(x -> quantile(x, 0.975), distr_α, dims=3)[:,:,1];

C_ind = 1;
yC_ind = 3;
τ_y = 5;
eC_ind = 6;
τ_e = 8;
uC_ind = 9;
τ_u = 11;



In [4]:


main_color = "#2A6EA6"                    # median line
band68     = "rgba(42,110,166,0.30)"      # inner band
band90     = "rgba(42,110,166,0.15)"      # outer band
data_color = "#222222"                    # raw data

# ============= Business Cyclical =============
plt_cycle = Plot([
    scatter(x=date, y=α_05[C_ind,:].*σʸ[1], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[C_ind,:].*σʸ[1], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[C_ind,:].*σʸ[1], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[C_ind,:].*σʸ[1], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[C_ind,:].*σʸ[1], mode="lines", name="Cyclical",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Output Gap", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=true),
    yaxis=attr(title="pct.", gridcolor="rgba(0,0,0,0.08)", zeroline=true),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_cycle)


# ============= Trend output + Data =============
plt_trend = Plot([
    # 90% band (lower then upper, fill to next y)
    scatter(x=date, y=α_05[τ_y,:].*σʸ[1], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[τ_y,:].*σʸ[1], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    # 68% band
    scatter(x=date, y=α_16[τ_y,:].*σʸ[1], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[τ_y,:].*σʸ[1], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    # median & data
    scatter(x=date, y=α_med[τ_y,:].*σʸ[1], mode="lines", name="Trend",
            line=attr(color=main_color, width=2.5)),
    scatter(x=date, y=y, mode="lines", name="Data",
            line=attr(color=data_color, width=1.4), opacity=0.9)
],
Layout(
    title="Trend Output", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_trend)

# ============= Idiosyncratic output =============
plt_idio = Plot([
    scatter(x=date, y=α_05[yC_ind,:].*σʸ[1], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[yC_ind,:].*σʸ[1], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[yC_ind,:].*σʸ[1], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[yC_ind,:].*σʸ[1], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[yC_ind,:].*σʸ[1], mode="lines", name="Idiosyncratic",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic Output Component", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="pct.", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))

# ============= Trend employment =============
plt_trend = Plot([
    # 90% band (lower then upper, fill to next y)
    scatter(x=date, y=α_05[τ_e,:].*σʸ[2], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[τ_e,:].*σʸ[2], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    # 68% band
    scatter(x=date, y=α_16[τ_e,:].*σʸ[2], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[τ_e,:].*σʸ[2], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    # median & data
    scatter(x=date, y=α_med[τ_e,:].*σʸ[2], mode="lines", name="Trend",
            line=attr(color=main_color, width=2.5)),
    scatter(x=date, y=e_data, mode="lines", name="Data",
            line=attr(color=data_color, width=1.4), opacity=0.9)
],
Layout(
    title="Trend Employment", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="Thousands", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_trend)

# ============= Idiosyncratic employment =============
plt_idio = Plot([
    scatter(x=date, y=α_05[eC_ind,:].*σʸ[2], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[eC_ind,:].*σʸ[2], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[eC_ind,:].*σʸ[2], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[eC_ind,:].*σʸ[2], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[eC_ind,:].*σʸ[2], mode="lines", name="Idiosyncratic",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic Employment Component", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="thousands", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_idio)

# ============= Trend Unemployment ============= 
plt_trend = Plot([
    # 90% band (lower then upper, fill to next y)
    scatter(x=date, y=α_05[τ_u,:].*σʸ[3], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[τ_u,:].*σʸ[3], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    # 68% band
    scatter(x=date, y=α_16[τ_u,:].*σʸ[3], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[τ_u,:].*σʸ[3], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    # median & data
    scatter(x=date, y=α_med[τ_u,:].*σʸ[3], mode="lines", name="Trend",
            line=attr(color=main_color, width=2.5)),
    scatter(x=date, y=u_data, mode="lines", name="Data",
            line=attr(color=data_color, width=1.4), opacity=0.9)
],
Layout(
    title="Trend Unemployment", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="Thousands", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_trend)

# ============= Idiosyncratic Unemployment =============
plt_idio = Plot([
    scatter(x=date, y=α_05[uC_ind,:].*σʸ[3], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[uC_ind,:].*σʸ[3], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[uC_ind,:].*σʸ[3], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[uC_ind,:].*σʸ[3], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[uC_ind,:].*σʸ[3], mode="lines", name="Idiosyncratic",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic Unemployment Component", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="thousands", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_idio)




data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, opacity, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, opacity, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, opacity, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"

